In [35]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = '/Users/arjunathreya/Downloads/listings.csv'
df = pd.read_csv(file_path)

# Display initial data information
print("Initial Data Overview:")
df.info()

Initial Data Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37541 entries, 0 to 37540
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            37541 non-null  int64  
 1   listing_url                                   37541 non-null  object 
 2   scrape_id                                     37541 non-null  int64  
 3   last_scraped                                  37541 non-null  object 
 4   source                                        37541 non-null  object 
 5   name                                          37539 non-null  object 
 6   description                                   36221 non-null  object 
 7   neighborhood_overview                         20891 non-null  object 
 8   picture_url                                   37540 non-null  object 
 9   host_id                               

In [36]:
# Data Exploration
missing_values = df.isnull().sum().sort_values(ascending=False)
missing_percentage = (missing_values / len(df)) * 100
print("\nMissing Values (%):")
print(pd.DataFrame({'Missing Values': missing_values, 'Percentage of missing values': missing_percentage}).head(10))


Missing Values (%):
                       Missing Values  Percentage of missing values
calendar_updated                37541                    100.000000
license                         32200                     85.772888
neighborhood_overview           16650                     44.351509
neighbourhood                   16649                     44.348845
host_about                      16247                     43.278016
host_response_rate              14897                     39.681948
host_response_time              14897                     39.681948
beds                            14886                     39.652646
price                           14751                     39.293040
bathrooms                       14751                     39.293040


In [37]:
print("\nBasic Statistics of Numeric Columns:")
print(df.describe())


Basic Statistics of Numeric Columns:
                 id     scrape_id       host_id  host_listings_count  \
count  3.754100e+04  3.754100e+04  3.754100e+04         37536.000000   
mean   3.783277e+17  2.024090e+13  1.682797e+08           210.164509   
std    4.643902e+17  0.000000e+00  1.820582e+08           830.651876   
min    2.595000e+03  2.024090e+13  1.678000e+03             1.000000   
25%    2.063928e+07  2.024090e+13  1.705622e+07             1.000000   
50%    4.866104e+07  2.024090e+13  8.415508e+07             2.000000   
75%    8.450413e+17  2.024090e+13  3.052402e+08             9.000000   
max    1.237702e+18  2.024090e+13  5.992925e+08          4494.000000   

       host_total_listings_count      latitude     longitude  accommodates  \
count               37536.000000  37541.000000  37541.000000  37541.000000   
mean                  294.218057     40.728722    -73.946982      2.760262   
std                  1050.908535      0.056340      0.054604      1.925977   
m

In [38]:
# Check for duplicates
duplicates = df.duplicated().sum()
print(f"\nNumber of duplicate rows: {duplicates}")


Number of duplicate rows: 0


In [39]:
# Decide on columns to keep
listing_df_cols = ['id', 'listing_url', 'price', 'property_type', 'room_type', 'neighborhood_overview', 'bathrooms_text', 'bedrooms', 'beds', 'accommodates', 'latitude', 'longitude', 'neighbourhood_group_cleansed', 'neighbourhood_cleansed', 'neighborhood_overview', 'description', 'amenities', 'host_about', 'review_scores_rating', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'number_of_reviews']
df = df[listing_df_cols]

In [40]:
# Handle Missing Values
# Define a threshold for dropping columns that have too many missing values (e.g., 50%)
threshold = 0.5 * len(df)
df_cleaned = df.dropna(thresh=threshold, axis=1)

print(f"\nColumns after dropping those with more than {threshold} missing values:")
print(df_cleaned.columns)

# Fill missing string/object columns with empty strings
string_cols = df_cleaned.select_dtypes(include=['object']).columns
for col in string_cols:
    df_cleaned[col] = df_cleaned[col].fillna('')

# Fill missing numerical columns with mean (for simplicity)
numeric_cols = df_cleaned.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_cols:
    df_cleaned[col] = df_cleaned[col].fillna(df_cleaned[col].mean())


Columns after dropping those with more than 18770.5 missing values:
Index(['id', 'listing_url', 'price', 'property_type', 'room_type',
       'neighborhood_overview', 'bathrooms_text', 'bedrooms', 'beds',
       'accommodates', 'latitude', 'longitude', 'neighbourhood_group_cleansed',
       'neighbourhood_cleansed', 'neighborhood_overview', 'description',
       'amenities', 'host_about', 'review_scores_rating',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'number_of_reviews'],
      dtype='object')


In [41]:
# Convert price column (remove symbols and commas) using .loc[]
if 'price' in df_cleaned.columns:
    df_cleaned.loc[:, 'price'] = df_cleaned['price'].replace('', np.nan)
    df_cleaned.loc[:, 'price'] = df_cleaned['price'].replace({'\$': '', ',': ''}, regex=True)
    df_cleaned.loc[:, 'price'] = pd.to_numeric(df_cleaned['price'], errors='coerce')

In [42]:
# Check for any remaining missing values
remaining_missing = df_cleaned.isnull().sum().sort_values(ascending=False)
remaining_missing_percentage = (remaining_missing / len(df_cleaned)) * 100
print("\nRemaining Missing Values after Cleanup:")
print(pd.DataFrame({'Missing Values': remaining_missing, 'Percentage': remaining_missing_percentage}).head(10))


Remaining Missing Values after Cleanup:
                             Missing Values  Percentage
price                                 14751    39.29304
id                                        0     0.00000
neighbourhood_cleansed                    0     0.00000
review_scores_value                       0     0.00000
review_scores_location                    0     0.00000
review_scores_communication               0     0.00000
review_scores_checkin                     0     0.00000
review_scores_cleanliness                 0     0.00000
review_scores_rating                      0     0.00000
host_about                                0     0.00000


In [43]:
# Check the data types after cleanup
print("\nData Types after Cleanup:")
print(df_cleaned.dtypes)


Data Types after Cleanup:
id                                int64
listing_url                      object
price                            object
property_type                    object
room_type                        object
neighborhood_overview            object
bathrooms_text                   object
bedrooms                        float64
beds                            float64
accommodates                      int64
latitude                        float64
longitude                       float64
neighbourhood_group_cleansed     object
neighbourhood_cleansed           object
neighborhood_overview            object
description                      object
amenities                        object
host_about                       object
review_scores_rating            float64
review_scores_cleanliness       float64
review_scores_checkin           float64
review_scores_communication     float64
review_scores_location          float64
review_scores_value             float64
number_of_rev

In [45]:
schema_columns = ['id', 'listing_url', 'neighborhood_overview', 'price', 'property_type', 'room_type', 'bathrooms_text', 'bedrooms', 'beds', 'accommodates', 'latitude', 'longitude', 'neighbourhood_group_cleansed', 'neighbourhood_cleansed', 'description', 'amenities', 'host_about', 'review_scores_rating', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'number_of_reviews']

# Handle missing prices
average_price = df_cleaned['price'].mean()
df_cleaned['price'] = df_cleaned['price'].fillna(average_price)

# Convert data types to match the schema
df_cleaned['id'] = df_cleaned['id'].astype(int)
df_cleaned['price'] = df_cleaned['price'].astype(float)
df_cleaned['bedrooms'] = df_cleaned['bedrooms'].astype(float)
df_cleaned['beds'] = df_cleaned['beds'].astype(float)
df_cleaned['accommodates'] = df_cleaned['accommodates'].astype(int)
df_cleaned['latitude'] = df_cleaned['latitude'].astype(float)
df_cleaned['longitude'] = df_cleaned['longitude'].astype(float)
df_cleaned['review_scores_rating'] = df_cleaned['review_scores_rating'].astype(float)
df_cleaned['review_scores_cleanliness'] = df_cleaned['review_scores_cleanliness'].astype(float)
df_cleaned['review_scores_checkin'] = df_cleaned['review_scores_checkin'].astype(float)
df_cleaned['review_scores_communication'] = df_cleaned['review_scores_communication'].astype(float)
df_cleaned['review_scores_location'] = df_cleaned['review_scores_location'].astype(float)
df_cleaned['review_scores_value'] = df_cleaned['review_scores_value'].astype(float)
df_cleaned['number_of_reviews'] = df_cleaned['number_of_reviews'].astype(int)

# Ensure text columns are not null
text_columns = ['listing_url', 'property_type', 'room_type', 'bathrooms_text', 'neighbourhood_group_cleansed', 'neighbourhood_cleansed', 'description', 'amenities', 'host_about']
for col in text_columns:
    df_cleaned[col] = df_cleaned[col].fillna('')

# Verify the changes
print("\nUpdated Data Types:")
print(df_cleaned.dtypes)

print("\nMissing Values After Modifications:")
print(df_cleaned.isnull().sum())


Updated Data Types:
id                                int64
listing_url                      object
price                           float64
property_type                    object
room_type                        object
neighborhood_overview            object
bathrooms_text                   object
bedrooms                        float64
beds                            float64
accommodates                      int64
latitude                        float64
longitude                       float64
neighbourhood_group_cleansed     object
neighbourhood_cleansed           object
neighborhood_overview            object
description                      object
amenities                        object
host_about                       object
review_scores_rating            float64
review_scores_cleanliness       float64
review_scores_checkin           float64
review_scores_communication     float64
review_scores_location          float64
review_scores_value             float64
number_of_reviews  

In [17]:
#save to csv
df_cleaned.to_csv('/Users/arjunathreya/Downloads/cleaned_listings.csv', index=False)